In [1]:
import tensorflow as tf 
 
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 텐서플로가 세 번째 GPU만 사용하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[3], 'GPU')
    except RuntimeError as e:
        # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
        print(e)

import pandas as pd
import json
import os 
import glob
from pandas import json_normalize
import numpy as np
from tensorflow import keras
from tensorflow.keras import *
from PIL import *
from tqdm import tqdm

import random 
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [2]:
target = r"C:/AIFactory/ETC/교통수신호/json/*"
target_json = glob.glob(target)

In [3]:
 b = []
for i in target_json:
    with open(i,"r") as j:
        contents = json.loads(j.read())
        a = json_normalize(contents)
        b.append(a)

In [4]:
train_json = pd.concat(b)

In [5]:
train_json1 = train_json.drop(columns = ['sequence.image_nums', 'sequence.2d_pos',
       'sequence.bounding_box', 'sequence.image_height',
       'sequence.image_width', 'info.actor_gender', 'info.actor_tall_range',
       'info.actor_cloth', 'info.s_height', 'info.lain_no', 'info.s_dynamic',
       'info.s_weather', 'info.s_season', 'info.direction'])

In [6]:
train_json2 = train_json1.reset_index(drop=True)
train_json2

,action
0,5
1,3
2,3
3,1
4,4
...,...
137,4
138,4
139,0
140,4


In [7]:
target_json = r"C:/AIFactory/ETC/교통수신호/json/*.json"
target_json1 = glob.glob(target_json)

json = pd.DataFrame({'path': target_json1})

In [8]:
json['split'] = json['path'].apply(lambda x : x.split('/')[-1].split('\\')[-1].split("_")[1].split(".")[0])
json

,path,split
0,C:/AIFactory/ETC/교통수신호/json\file_0.json,0
1,C:/AIFactory/ETC/교통수신호/json\file_1.json,1
2,C:/AIFactory/ETC/교통수신호/json\file_10.json,10
3,C:/AIFactory/ETC/교통수신호/json\file_100.json,100
4,C:/AIFactory/ETC/교통수신호/json\file_101.json,101
...,...,...
137,C:/AIFactory/ETC/교통수신호/json\file_95.json,95
138,C:/AIFactory/ETC/교통수신호/json\file_96.json,96
139,C:/AIFactory/ETC/교통수신호/json\file_97.json,97
140,C:/AIFactory/ETC/교통수신호/json\file_98.json,98


In [9]:
json1 = pd.concat([json,train_json2],axis=1)
json2 = json1.drop(columns = ['path'])

In [10]:
json2

,split,action
0,0,5
1,1,3
2,10,3
3,100,1
4,101,4
...,...,...
137,95,4
138,96,4
139,97,0
140,98,4


In [11]:
test_target = r"C:/AIFactory/ETC/교통수신호/new_images/test/**/*.jpg"
test_target_png = glob.glob(test_target)

test = pd.DataFrame({'path': test_target_png})
test.head(3)

,path
0,C:/AIFactory/ETC/교통수신호/new_images/test\file_14...
1,C:/AIFactory/ETC/교통수신호/new_images/test\file_14...
2,C:/AIFactory/ETC/교통수신호/new_images/test\file_14...


In [12]:
target = r"C:/AIFactory/ETC/교통수신호/new_images/train/**/*.jpg"
target_jpg = glob.glob(target)
train = pd.DataFrame({'path': target_jpg})
train.head(3)

,path
0,C:/AIFactory/ETC/교통수신호/new_images/train\file_0...
1,C:/AIFactory/ETC/교통수신호/new_images/train\file_0...
2,C:/AIFactory/ETC/교통수신호/new_images/train\file_0...


In [13]:
train['split'] = train['path'].apply(lambda x : x.split('/')[-1].split('\\')[1].split('_')[-1])
train

,path,split
0,C:/AIFactory/ETC/교통수신호/new_images/train\file_0...,0
1,C:/AIFactory/ETC/교통수신호/new_images/train\file_0...,0
2,C:/AIFactory/ETC/교통수신호/new_images/train\file_0...,0
3,C:/AIFactory/ETC/교통수신호/new_images/train\file_0...,0
4,C:/AIFactory/ETC/교통수신호/new_images/train\file_0...,0
...,...,...
22213,C:/AIFactory/ETC/교통수신호/new_images/train\file_9...,99
22214,C:/AIFactory/ETC/교통수신호/new_images/train\file_9...,99
22215,C:/AIFactory/ETC/교통수신호/new_images/train\file_9...,99
22216,C:/AIFactory/ETC/교통수신호/new_images/train\file_9...,99


In [14]:
train['split'] = train['split'].astype('int')
json2['split'] = json2['split'].astype('int')

alldata = pd.merge(train,json2, how ='left', on ='split')

alldata1=alldata.drop_duplicates()

In [15]:
alldata1

,path,split,action
0,C:/AIFactory/ETC/교통수신호/new_images/train\file_0...,0,5
1,C:/AIFactory/ETC/교통수신호/new_images/train\file_0...,0,5
2,C:/AIFactory/ETC/교통수신호/new_images/train\file_0...,0,5
3,C:/AIFactory/ETC/교통수신호/new_images/train\file_0...,0,5
4,C:/AIFactory/ETC/교통수신호/new_images/train\file_0...,0,5
...,...,...,...
22213,C:/AIFactory/ETC/교통수신호/new_images/train\file_9...,99,4
22214,C:/AIFactory/ETC/교통수신호/new_images/train\file_9...,99,4
22215,C:/AIFactory/ETC/교통수신호/new_images/train\file_9...,99,4
22216,C:/AIFactory/ETC/교통수신호/new_images/train\file_9...,99,4


In [16]:
alldata1['action'] =  alldata1['action'].astype('str')  

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
from tensorflow.keras.applications.efficientnet import EfficientNetB2
from tensorflow.keras.callbacks  import *
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.applications.vgg19 import VGG19

In [21]:
skf = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 20)
result = 0

In [25]:
for train_index, valid_index in tqdm(skf.split(alldata1,alldata1['action'])):
    x_train = alldata1.iloc[train_index]
    x_valid = alldata1.iloc[valid_index]
    idg = ImageDataGenerator(horizontal_flip = True)
    idg2 = ImageDataGenerator()
    train_generator = idg.flow_from_dataframe(x_train, x_col = 'path', y_col = 'action', target_size = (300,300), batch_size = 64)
    valid_generator = idg2.flow_from_dataframe(x_valid, x_col = 'path', y_col = 'action', target_size = (300,300), batch_size = 64)
    es = EarlyStopping(patience = 3, restore_best_weights=True, verbose = 1)

    rl = ReduceLROnPlateau(patience = 2, factor = 0.2, verbose = 1)

    vgg = VGG19(input_shape = (224,224,3),include_top = False, weights='imagenet')
    vgg.trainable = False
    

    model = Sequential()

    model.add(vgg)
    model.add(AveragePooling2D(pool_size=(7, 7)))
    model.add(Flatten(name="flatten"))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation = 'relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(6, activation = 'softmax'))
    model.compile(metrics = ['acc'], loss = 'categorical_crossentropy', optimizer = 'adam')

    model.fit(train_generator, validation_data = valid_generator, epochs = 20, callbacks = [es,rl])
    
    test_generator = idg2.flow_from_dataframe(test, x_col = 'path', y_col = None, target_size = (300,300), batch_size = 64, class_mode = None, shuffle = False) 
    
    result += model.predict(test_generator, verbose = 1, workers=2) / 3

0it [00:00, ?it/s]

Found 14812 validated image filenames belonging to 6 classes.
Found 7406 validated image filenames belonging to 6 classes.
Epoch 1/20
232/232 [==============================] - 42s 180ms/step - loss: 2.0720 - acc: 0.2055 - val_loss: 1.5176 - val_acc: 0.3681
Epoch 2/20
232/232 [==============================] - 42s 179ms/step - loss: 1.5286 - acc: 0.3448 - val_loss: 0.9019 - val_acc: 0.7120
Epoch 3/20
232/232 [==============================] - 42s 179ms/step - loss: 1.0805 - acc: 0.5790 - val_loss: 0.5542 - val_acc: 0.8381
Epoch 4/20
232/232 [==============================] - 42s 179ms/step - loss: 0.7834 - acc: 0.7156 - val_loss: 0.3748 - val_acc: 0.8833
Epoch 5/20
232/232 [==============================] - 42s 182ms/step - loss: 0.5510 - acc: 0.8050 - val_loss: 0.2413 - val_acc: 0.9317
Epoch 6/20
232/232 [==============================] - 42s 179ms/step - loss: 0.4586 - acc: 0.8433 - val_loss: 0.1517 - val_acc: 0.9584
Epoch 7/20
232/232 [==============================] - 42s 180ms/ste

1it [15:32, 932.54s/it]

Found 14812 validated image filenames belonging to 6 classes.
Found 7406 validated image filenames belonging to 6 classes.
Epoch 1/20
232/232 [==============================] - 62s 265ms/step - loss: 1.9941 - acc: 0.1843 - val_loss: 1.4930 - val_acc: 0.4895
Epoch 2/20
232/232 [==============================] - 63s 272ms/step - loss: 1.5021 - acc: 0.3925 - val_loss: 0.9748 - val_acc: 0.6839acc
Epoch 3/20
232/232 [==============================] - 64s 277ms/step - loss: 1.0693 - acc: 0.6029 - val_loss: 0.6184 - val_acc: 0.8184
Epoch 4/20
232/232 [==============================] - 68s 292ms/step - loss: 0.7899 - acc: 0.7141 - val_loss: 0.3955 - val_acc: 0.8771
Epoch 5/20
232/232 [==============================] - 69s 299ms/step - loss: 0.6020 - acc: 0.7959 - val_loss: 0.2978 - val_acc: 0.9159
Epoch 6/20
232/232 [==============================] - 72s 310ms/step - loss: 0.5023 - acc: 0.8368 - val_loss: 0.2063 - val_acc: 0.9437
Epoch 7/20
232/232 [==============================] - 74s 319ms/

2it [42:44, 1142.32s/it]

Found 14812 validated image filenames belonging to 6 classes.
Found 7406 validated image filenames belonging to 6 classes.
Epoch 1/20
232/232 [==============================] - 102s 435ms/step - loss: 1.9523 - acc: 0.2227 - val_loss: 1.2967 - val_acc: 0.5257
Epoch 2/20
232/232 [==============================] - 101s 437ms/step - loss: 1.3539 - acc: 0.4302 - val_loss: 0.8028 - val_acc: 0.7571
Epoch 3/20
232/232 [==============================] - 105s 451ms/step - loss: 0.9785 - acc: 0.6106 - val_loss: 0.4924 - val_acc: 0.8517
Epoch 4/20
232/232 [==============================] - 105s 452ms/step - loss: 0.7261 - acc: 0.7221 - val_loss: 0.2919 - val_acc: 0.9128
Epoch 5/20
232/232 [==============================] - 106s 455ms/step - loss: 0.5445 - acc: 0.8066 - val_loss: 0.2572 - val_acc: 0.9257
Epoch 6/20
232/232 [==============================] - 110s 475ms/step - loss: 0.4550 - acc: 0.8319 - val_loss: 0.1753 - val_acc: 0.9519
Epoch 7/20
232/232 [==============================] - 114s 49

3it [1:23:24, 1668.10s/it]


In [26]:
answer = []
for i in range(len(test)):
    answer.append(result[i])

In [27]:
sample = pd.read_csv("sample_submission.csv")

In [28]:
sub = pd.DataFrame(columns = sample.columns, index=test.index)
sub['file_path'] = test['path']

In [29]:
sub.iloc[:,1:] = answer

In [41]:
sub.to_csv('vgg19_raw.csv',index=False)

In [30]:
sub['label'] = sub['file_path'].apply(lambda x : x.split('/')[-1].split('\\')[1].split('_')[-1])

In [36]:
sub1 = sub.drop(columns = ['file_path'])
sub2 = sub1.groupby('label').agg(['mean']).reset_index(drop=True)

In [37]:
sample.iloc[:,1:] = sub2
sample

,file_path,Label_0,Label_1,Label_2,Label_3,Label_4,Label_5
0,./test\file_142,0.042573,0.015164,0.284463,0.440909,0.043725,0.173165
1,./test\file_143,0.005516,0.010050,0.131265,0.023309,0.014463,0.815397
2,./test\file_144,0.003039,0.019211,0.086748,0.866482,0.001662,0.022857
3,./test\file_145,0.001361,0.002368,0.867653,0.004216,0.002653,0.121749
4,./test\file_146,0.945103,0.035472,0.004845,0.000927,0.006657,0.006996
5,./test\file_147,0.806255,0.001701,0.004493,0.000329,0.185430,0.001792
6,./test\file_148,0.609421,0.111545,0.011294,0.112703,0.139089,0.015948
7,./test\file_149,0.015251,0.080901,0.006114,0.049099,0.844754,0.003881
8,./test\file_150,0.020460,0.794895,0.024330,0.036800,0.037485,0.086030
9,./test\file_151,0.008694,0.047912,0.009574,0.001590,0.005379,0.926851


In [38]:
sample.to_csv('sub3.csv',index=False)

In [39]:
label_list = list(train_generator.class_indices.keys())

a = []
for i in range(sub.shape[0]):
    a.append(label_list[list(answer[i]).index(max(answer[i]))])


In [35]:
a

['3',
 '3',
 '3',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '3',
 '2',
 '2',
 '2',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '5',
 '3',
 '3',
 '3',
 '3',
 '3',
 '2',
 '2',
 '3',
 '3',
 '2',
 '2',
 '2',
 '2',
 '2',
 '3',
 '3',
 '2',
 '3',
 '5',
 '3',
 '5',
 '3',
 '2',
 '2',
 '3',
 '5',
 '5',
 '3',
 '0',
 '0',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '2',
 '0',
 '3',
 '3',
 '3',
 '2',
 '3',
 '2',
 '3',
 '3',
 '2',
 '2',
 '2',
 '2',
 '3',
 '2',
 '3',
 '2',
 '2',
 '2',
 '3',
 '2',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '2',
 '2',
 '2',
 '2',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '2',
 '3',
 '3',
 '5',
 '3',
 '5',
 '5',
 '3',
 '3',
 '3',
 '3',
 '4',
 '4',
 '4',
 '0',
 '5',
 '5',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '5',
 '5',
 '5',
 '5',
 '5',
 '3',
 '0',
 '0',
 '3',
 '3',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '5',
 '5',
 '5',
 '5',
 '5',
 '2',
 '5',
 '5',
 '2',
 '2',
 '5',
 '5'